## Imports and CSV Reading

In [503]:
# Import necessary libraries
import math
import statistics as stat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector

# Load the dataset
file_path = "../Data/Latest_Data_Science_Salaries.csv"
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
df.head()

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
0,Data Engineer,Full-Time,Senior,Expert,210000,United States Dollar,United States,210000,United States,Medium,2023
1,Data Engineer,Full-Time,Senior,Expert,165000,United States Dollar,United States,165000,United States,Medium,2023
2,Data Engineer,Full-Time,Senior,Expert,185900,United States Dollar,United States,185900,United States,Medium,2023
3,Data Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2023
4,Data Scientist,Full-Time,Senior,Expert,140000,United States Dollar,United States,140000,United States,Medium,2023


## Clean the Data

In [396]:
# Remove values of Employment Type that are not listed as Full-Time
# Remove values of Company Location that are not listed as United States
new_df = df[(df['Employment Type'] == 'Full-Time') & (df['Employee Residence'] == 'United States')]
new_df = new_df.reset_index(drop = True)
new_df

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
0,Data Engineer,Full-Time,Senior,Expert,210000,United States Dollar,United States,210000,United States,Medium,2023
1,Data Engineer,Full-Time,Senior,Expert,165000,United States Dollar,United States,165000,United States,Medium,2023
2,Data Engineer,Full-Time,Senior,Expert,185900,United States Dollar,United States,185900,United States,Medium,2023
3,Data Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2023
4,Data Scientist,Full-Time,Senior,Expert,140000,United States Dollar,United States,140000,United States,Medium,2023
...,...,...,...,...,...,...,...,...,...,...,...
2437,Applied Machine Learning Scientist,Full-Time,Mid,Intermediate,423000,United States Dollar,United States,423000,United States,Large,2021
2438,Data Specialist,Full-Time,Senior,Expert,165000,United States Dollar,United States,165000,United States,Large,2021
2439,Data Scientist,Full-Time,Senior,Expert,412000,United States Dollar,United States,412000,United States,Large,2020
2440,Principal Data Scientist,Full-Time,Mid,Intermediate,151000,United States Dollar,United States,151000,United States,Large,2021


In [398]:
# Drop Expertise Level, Salary Currency, and Salary in USD columns
df_clean = new_df.drop(columns = ['Employment Type', 'Expertise Level', 'Salary Currency', 'Employee Residence', 'Salary in USD'])
df_clean

,Job Title,Experience Level,Salary,Company Location,Company Size,Year
0,Data Engineer,Senior,210000,United States,Medium,2023
1,Data Engineer,Senior,165000,United States,Medium,2023
2,Data Engineer,Senior,185900,United States,Medium,2023
3,Data Engineer,Senior,129300,United States,Medium,2023
4,Data Scientist,Senior,140000,United States,Medium,2023
...,...,...,...,...,...,...
2437,Applied Machine Learning Scientist,Mid,423000,United States,Large,2021
2438,Data Specialist,Senior,165000,United States,Large,2021
2439,Data Scientist,Senior,412000,United States,Large,2020
2440,Principal Data Scientist,Mid,151000,United States,Large,2021


In [399]:
# Check the total number of employees working for companies located in the United States
(df_clean['Company Location'] == 'United States').value_counts()

True     2437
False       5
Name: Company Location, dtype: int64

In [400]:
# Check to see how many employees residing in the United States do not work full-time
((df['Employment Type'] != 'Full-Time') & (df['Employee Residence'] == 'United States')).value_counts()

False    3289
True       11
dtype: int64

In [244]:
# # Split job titles to look at last word in job title
# len_jt = range(2475)

# for i in len_jt:
#     last_word = df_clean['Job Title'][i].split()[-1]
#     df_clean.at[i, 'Job Title'] = last_word

In [401]:
titles_counts = df_clean['Job Title'].value_counts()
titles_counts.head(20)

Data Engineer                     554
Data Scientist                    463
Data Analyst                      355
Machine Learning Engineer         214
Analytics Engineer                108
Research Scientist                 80
Data Architect                     76
Data Science Manager               53
ML Engineer                        48
Applied Scientist                  47
Research Engineer                  46
Machine Learning Scientist         35
Data Manager                       27
Data Analytics Manager             21
Business Intelligence Engineer     20
Data Specialist                    18
BI Developer                       16
Data Science Consultant            16
BI Analyst                         13
Computer Vision Engineer           11
Name: Job Title, dtype: int64

In [402]:
# Choose a cutoff value and create a list of job titles to be replaced
# use the variable name `residence_to_replace`
cutoff_value = 50

titles_to_replace = titles_counts[titles_counts < cutoff_value].index.tolist()

# Replace in dataframe
for jt in titles_to_replace:
    df_clean['Job Title'] = df_clean['Job Title'].replace(jt, "Other")

# Check to make sure binning was successful
df_clean['Job Title'].value_counts()

Data Engineer                554
Other                        539
Data Scientist               463
Data Analyst                 355
Machine Learning Engineer    214
Analytics Engineer           108
Research Scientist            80
Data Architect                76
Data Science Manager          53
Name: Job Title, dtype: int64

In [403]:
# Check to see how many companies are located outside of the United States
location_counts = df_clean['Company Location'].value_counts()
location_counts

United States    2437
Japan               1
Australia           1
Germany             1
Canada              1
France              1
Name: Company Location, dtype: int64

In [404]:
# Choose a cutoff value and create a list of countries of residence to be replaced
# use the variable name `residence_to_replace`
cutoff_value = 1000

location_to_replace = location_counts[location_counts < cutoff_value].index.tolist()

# Replace in dataframe
for loca in location_to_replace:
    df_clean['Company Location'] = df_clean['Company Location'].replace(loca, "Other")

# Check to make sure binning was successful
df_clean['Company Location'].value_counts()

United States    2437
Other               5
Name: Company Location, dtype: int64

## Encoding Qualitative Variables

In [405]:
# Display cleaned dataframe
df_clean

,Job Title,Experience Level,Salary,Company Location,Company Size,Year
0,Data Engineer,Senior,210000,United States,Medium,2023
1,Data Engineer,Senior,165000,United States,Medium,2023
2,Data Engineer,Senior,185900,United States,Medium,2023
3,Data Engineer,Senior,129300,United States,Medium,2023
4,Data Scientist,Senior,140000,United States,Medium,2023
...,...,...,...,...,...,...
2437,Other,Mid,423000,United States,Large,2021
2438,Other,Senior,165000,United States,Large,2021
2439,Data Scientist,Senior,412000,United States,Large,2020
2440,Other,Mid,151000,United States,Large,2021


In [406]:
# Display value counts of job titles
df_clean['Job Title'].value_counts()

Data Engineer                554
Other                        539
Data Scientist               463
Data Analyst                 355
Machine Learning Engineer    214
Analytics Engineer           108
Research Scientist            80
Data Architect                76
Data Science Manager          53
Name: Job Title, dtype: int64

In [407]:
# Code job titles into numeric values
df_clean.loc[df_clean['Job Title'] == 'Data Engineer', 'Job Title'] = 0
df_clean.loc[df_clean['Job Title'] == 'Data Scientist', 'Job Title'] = 1
df_clean.loc[df_clean['Job Title'] == 'Data Analyst', 'Job Title'] = 2
df_clean.loc[df_clean['Job Title'] == 'Machine Learning Engineer', 'Job Title'] = 3
df_clean.loc[df_clean['Job Title'] == 'Analytics Engineer', 'Job Title'] = 4
df_clean.loc[df_clean['Job Title'] == 'Research Scientist', 'Job Title'] = 5
df_clean.loc[df_clean['Job Title'] == 'Data Architect', 'Job Title'] = 6
df_clean.loc[df_clean['Job Title'] == 'Data Science Manager', 'Job Title'] = 7
df_clean.loc[df_clean['Job Title'] == 'Other', 'Job Title'] = 8

In [408]:
# Display value counts of experience levels
df_clean['Experience Level'].value_counts()

Senior       1734
Mid           440
Entry         151
Executive     117
Name: Experience Level, dtype: int64

In [409]:
# Code experience levels into numeric values
df_clean.loc[df_clean['Experience Level'] == 'Entry', 'Experience Level'] = 0
df_clean.loc[df_clean['Experience Level'] == 'Mid', 'Experience Level'] = 1
df_clean.loc[df_clean['Experience Level'] == 'Senior', 'Experience Level'] = 2
df_clean.loc[df_clean['Experience Level'] == 'Executive', 'Experience Level'] = 3

In [410]:
# Code company location into numeric values
df_clean.loc[df_clean['Company Location'] == 'United States', 'Company Location'] = 0
df_clean.loc[df_clean['Company Location'] == 'Other', 'Company Location'] = 1

In [411]:
# Display company sizes
df_clean['Company Size'].value_counts()

Medium    2176
Large      230
Small       36
Name: Company Size, dtype: int64

In [412]:
# Code company size into numeric values
df_clean.loc[df_clean['Company Size'] == 'Small', 'Company Size'] = 0
df_clean.loc[df_clean['Company Size'] == 'Medium', 'Company Size'] = 1
df_clean.loc[df_clean['Company Size'] == 'Large', 'Company Size'] = 2

## Linear Regression

In [413]:
# Display cleaned dataframe with encoded variables
df_clean

,Job Title,Experience Level,Salary,Company Location,Company Size,Year
0,0,2,210000,0,1,2023
1,0,2,165000,0,1,2023
2,0,2,185900,0,1,2023
3,0,2,129300,0,1,2023
4,1,2,140000,0,1,2023
...,...,...,...,...,...,...
2437,8,1,423000,0,2,2021
2438,8,2,165000,0,2,2021
2439,1,2,412000,0,2,2020
2440,8,1,151000,0,2,2021


In [414]:
# Store the independent variable in y and create a dataframe with the dependent variables
y = df_clean['Salary']
X = df_clean.drop(columns = ['Salary'])

# Display dataframe
X

,Job Title,Experience Level,Company Location,Company Size,Year
0,0,2,0,1,2023
1,0,2,0,1,2023
2,0,2,0,1,2023
3,0,2,0,1,2023
4,1,2,0,1,2023
...,...,...,...,...,...
2437,8,1,0,2,2021
2438,8,2,0,2,2021
2439,1,2,0,2,2020
2440,8,1,0,2,2021


In [415]:
y.value_counts()

150000    46
200000    42
100000    42
120000    39
130000    33
          ..
269600     1
250500     1
159500     1
94035      1
412000     1
Name: Salary, Length: 895, dtype: int64

In [416]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [423]:
# Fit the multiple linear regression model
mlr_model = linear_model.LinearRegression()
mlr_model.fit(X_train, y_train)

# Store the predicted 
y_test = y_test.reset_index(drop = True)
y_pred = mlr_model.predict(X_test)

array([  2446.92828329,  33736.17530884, -24264.55587609,  10547.83069734,
         8413.25710345])

In [428]:
# Create predictions and residuals dataframe
pred_df = pd.DataFrame(y_pred, columns = ['Predictions'])
pred_df['Testing Data'] = y_test
residuals = y_test - y_pred
pred_df['Residuals'] = residuals
pred_df

,Predictions,Testing Data,Residuals
0,183449.276602,140000,-43449.276602
1,88602.581081,90000,1397.418919
2,157907.521516,247500,89592.478484
3,160354.449799,123648,-36706.449799
4,171214.635186,311000,139785.364814
...,...,...,...
606,130137.675027,106500,-23637.675027
607,183449.276602,225000,41550.723398
608,157907.521516,149850,-8057.521516
609,177170.593093,165000,-12170.593093


In [430]:
# Calculate the R^2 value and display residuals vs. fitted plot
print(r2_score(y_test, y_pred))
pred_df.hvplot.scatter(x = 'Predictions', y = 'Residuals', title = 'Residuals vs. Fitted Plot')

0.07532328882365014


:Scatter   [Predictions]   (Residuals)

The residuals vs. fitted graph using the model with all variables shows heteroscedasticity, meaning the variance of the standard error, or y-intercept, is not constant throughout the model. This is an issue because it invalidates tests of statistical significance. In order to correct this we used stepwise variable selection with Akaike Information Criterion and Bayesian Information Criterion in order to test if our model could be improved through simplifying our model.

## Model Selection

In [581]:
# Calculate AIC and BIC for original model
mse1 = stat.mean(residuals ** 2)
n1 = len(residuals)
k1 = (len(mlr_model.coef_) + 1)
aic1 = (2*k1 + n1*math.log(mse1) + n1*math.log(2*math.pi) + n1)
bic1 = (k1*math.log(n1) + n1*math.log(mse1) + n1*math.log(2*math.pi) + n1)
print('AIC is', aic1)
print('BIC is', bic1)

AIC is 15171.620568383018
BIC is 15198.111150138047


In [582]:
# Set up for AIC and BIC function
mod_col = X.columns.tolist()
col_range = range(len(mod_col))
AIC_list = []
BIC_list = []

In [583]:
# Create function for calculating AIC and BIC
for col in col_range:
    X2 = X.drop(columns = mod_col[col])
    X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, random_state = 1)
    y2_test = y2_test.reset_index(drop = True)
    mlr_model2 = linear_model.LinearRegression()
    mlr_model2.fit(X2_train, y2_train)
    y2_pred = mlr_model2.predict(X2_test)
    residuals2 = y2_test - y2_pred
    mse = stat.mean(residuals2 ** 2)
    n = len(residuals2)
    k = (len(mlr_model2.coef_) + 1)
    aic = (2*k + n*math.log(mse) + n*math.log(2*math.pi) + n)
    bic = (k*math.log(n) + n*math.log(mse) + n*math.log(2*math.pi) + n)
    AIC_list.append(aic)
    BIC_list.append(bic)

In [584]:
# Print AIC and BIC results
# Compare to model with no dropped variables
print(AIC_list)
print(BIC_list)

[15167.177641529357, 15218.974214600446, 15168.79666909587, 15172.122654980802, 15169.892897436006]
[15189.253126325215, 15241.049699396304, 15190.872153891727, 15194.19813977666, 15191.968382231864]


In [590]:
# Display dataframe
X

,Job Title,Experience Level,Company Location,Company Size,Year
0,0,2,0,1,2023
1,0,2,0,1,2023
2,0,2,0,1,2023
3,0,2,0,1,2023
4,1,2,0,1,2023
...,...,...,...,...,...
2437,8,1,0,2,2021
2438,8,2,0,2,2021
2439,1,2,0,2,2020
2440,8,1,0,2,2021


In [591]:
# Drop variable with the lowest AIC and BIC
X2 = X.drop(columns = ['Job Title'])
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, random_state = 1)
y2_test = y2_test.reset_index(drop = True)
mlr_model2 = linear_model.LinearRegression()
mlr_model2.fit(X2_train, y2_train)
y2_pred = mlr_model2.predict(X2_test)
residuals2 = y2_test - y2_pred


In [592]:
# Create predictions and residuals dataframe
pred2_df = pd.DataFrame(y2_pred, columns = ['Predictions'])
pred2_df['Testing Data'] = y2_test
pred2_df['Residuals'] = residuals2
pred2_df

,Predictions,Testing Data,Residuals
0,171718.312486,140000,-31718.312486
1,73310.629287,90000,16689.370713
2,162596.882563,247500,84903.117437
3,162596.882563,123648,-38948.882563
4,171718.312486,311000,139281.687514
...,...,...,...
606,138118.863592,106500,-31618.863592
607,171718.312486,225000,53281.687514
608,162596.882563,149850,-12746.882563
609,166441.378206,165000,-1441.378206


In [593]:
# Calculate the R^2 value and display residuals vs. fitted plot
print(r2_score(y2_test, y2_pred))
pred2_df.hvplot.scatter(x = 'Predictions', y = 'Residuals', title = 'Residuals vs. Fitted Plot')

0.07901299050905852


:Scatter   [Predictions]   (Residuals)

In [594]:
# Calculate AIC for second model
mse2 = stat.mean(residuals2 ** 2)
n2 = len(residuals2)
k2 = (len(mlr_model2.coef_) + 1)
aic2 = (2*k2 + n2*math.log(mse2) + n2*math.log(2*math.pi) + n2)
bic2 = (k2*math.log(n2) + n2*math.log(mse2) + n2*math.log(2*math.pi) + n2)
print('AIC is', aic2)
print('BIC is', bic2)

AIC is 15167.177641529357
BIC is 15189.253126325215


In [595]:
# Set up for AIC and BIC function
mod_col2 = X2.columns.tolist()
col_range2 = range(len(mod_col2))
AIC_list2 = []
BIC_list2 = []

In [596]:
# Calculate AIC for possible models
for col in col_range2:
    X3 = X2.drop(columns = mod_col2[col])
    X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y, random_state = 1)
    y3_test = y3_test.reset_index(drop = True)
    mlr_model3 = linear_model.LinearRegression()
    mlr_model3.fit(X3_train, y3_train)
    y3_pred = mlr_model3.predict(X3_test)
    residuals3 = y3_test - y3_pred
    mse = stat.mean(residuals3 ** 2)
    n = len(residuals3)
    k = (len(mlr_model3.coef_) + 1)
    aic = (2*k + n*math.log(mse) + n*math.log(2*math.pi) + n)
    bic = (k*math.log(n) + n*math.log(mse) + n*math.log(2*math.pi) + n)
    AIC_list2.append(aic)
    BIC_list2.append(bic)

In [597]:
# Display AIC and BIC scores to determine which variable to drop
print(AIC_list2)
print(BIC_list2)

[15214.431647710726, 15164.300128825485, 15167.315386341219, 15164.247192714174]
[15232.092035547412, 15181.960516662171, 15184.975774177905, 15181.90758055086]


In [589]:
# Display dataframe
X2

,Job Title,Experience Level,Company Location,Company Size
0,0,2,0,1
1,0,2,0,1
2,0,2,0,1
3,0,2,0,1
4,1,2,0,1
...,...,...,...,...
2437,8,1,0,2
2438,8,2,0,2
2439,1,2,0,2
2440,8,1,0,2


In [598]:
# Drop variable with the lowest AIC and BIC
X3 = X2.drop(columns = ['Year'])
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y, random_state = 1)
y3_test = y3_test.reset_index(drop = True)
mlr_model3 = linear_model.LinearRegression()
mlr_model3.fit(X3_train, y3_train)
y3_pred = mlr_model3.predict(X3_test)
residuals3 = y3_test - y3_pred

In [599]:
# Create predictions and residuals dataframe
pred3_df = pd.DataFrame(y3_pred, columns = ['Predictions'])
pred3_df['Testing Data'] = y3_test
pred3_df['Residuals'] = residuals3
pred3_df

,Predictions,Testing Data,Residuals
0,168632.719711,140000,-28632.719711
1,91220.346196,90000,-1220.346196
2,168632.719711,247500,78867.280289
3,168632.719711,123648,-44984.719711
4,168632.719711,311000,142367.280289
...,...,...,...
606,134055.352579,106500,-27555.352579
607,168632.719711,225000,56367.280289
608,168632.719711,149850,-18782.719711
609,176890.358962,165000,-11890.358962


In [600]:
# Calculate the R^2 value and display residuals vs. fitted plot
print(r2_score(y3_test, y3_pred))
pred3_df.hvplot.scatter(x = 'Predictions', y = 'Residuals', title = 'Residuals vs. Fitted Plot')

0.08041442933517517


:Scatter   [Predictions]   (Residuals)

In [601]:
# Calculate AIC for third model
mse3 = stat.mean(residuals3 ** 2)
n3 = len(residuals3)
k3 = (len(mlr_model3.coef_) + 1)
aic3 = (2*k3 + n3*math.log(mse3) + n3*math.log(2*math.pi) + n3)
bic3 = (k3*math.log(n3) + n3*math.log(mse3) + n3*math.log(2*math.pi) + n3)
print('AIC is', aic3)
print('BIC is', bic3)

AIC is 15164.247192714174
BIC is 15181.90758055086


In [602]:
# Set up for AIC and BIC function
mod_col3 = X3.columns.tolist()
col_range3 = range(len(mod_col3))
AIC_list3 = []
BIC_list3 = []

In [603]:
# Calculate AIC and BIC for possible models
for col in col_range3:
    X4 = X3.drop(columns = mod_col3[col])
    X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y, random_state = 1)
    y4_test = y4_test.reset_index(drop = True)
    mlr_model4 = linear_model.LinearRegression()
    mlr_model4.fit(X4_train, y4_train)
    y4_pred = mlr_model4.predict(X4_test)
    residuals4 = y4_test - y4_pred
    mse = stat.mean(residuals4 ** 2)
    n = len(residuals4)
    k = (len(mlr_model4.coef_) + 1)
    aic = (2*k + n*math.log(mse) + n*math.log(2*math.pi) + n)
    bic = (k*math.log(n) + n*math.log(mse) + n*math.log(2*math.pi) + n)
    AIC_list3.append(aic)
    BIC_list3.append(bic)

In [604]:
# Display AIC and BIC scores to determine which variable to drop
print(AIC_list3)
print(BIC_list3)

[15214.316078385396, 15161.408936102831, 15163.883006045899]
[15227.56136926291, 15174.654226980345, 15177.128296923413]


In [605]:
# Display dataframe
X3

,Experience Level,Company Location,Company Size
0,2,0,1
1,2,0,1
2,2,0,1
3,2,0,1
4,2,0,1
...,...,...,...
2437,1,0,2
2438,2,0,2
2439,2,0,2
2440,1,0,2


In [606]:
# Drop variable with the lowest AIC and BIC
X4 = X3.drop(columns = ['Company Location'])
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y, random_state = 1)
y4_test = y4_test.reset_index(drop = True)
mlr_model4 = linear_model.LinearRegression()
mlr_model4.fit(X4_train, y4_train)
y4_pred = mlr_model4.predict(X4_test)
residuals4 = y4_test - y4_pred

In [607]:
# Create predictions and residuals dataframe
pred4_df = pd.DataFrame(y4_pred, columns = ['Predictions'])
pred4_df['Testing Data'] = y4_test
pred4_df['Residuals'] = residuals4
pred4_df

,Predictions,Testing Data,Residuals
0,168612.317188,140000,-28612.317188
1,91537.621572,90000,-1537.621572
2,168612.317188,247500,78887.682812
3,168612.317188,123648,-44964.317188
4,168612.317188,311000,142387.682812
...,...,...,...
606,134018.480138,106500,-27518.480138
607,168612.317188,225000,56387.682812
608,168612.317188,149850,-18762.317188
609,176499.338704,165000,-11499.338704


In [608]:
# Calculate the R^2 value and display residuals vs. fitted plot
print(r2_score(y4_test, y4_pred))
pred4_df.hvplot.scatter(x = 'Predictions', y = 'Residuals', title = 'Residuals vs. Fitted Plot')

0.0816751824385592


:Scatter   [Predictions]   (Residuals)

In [609]:
# Calculate AIC and BIC for fourth model
mse4 = stat.mean(residuals4 ** 2)
n4 = len(residuals4)
k4 = (len(mlr_model4.coef_) + 1)
aic4 = (2*k4 + n4*math.log(mse4) + n4*math.log(2*math.pi) + n4)
bic4 = (k4*math.log(n4) + n4*math.log(mse4) + n4*math.log(2*math.pi) + n4)
print('AIC is', aic4)
print('BIC is', bic4)

AIC is 15161.408936102831
BIC is 15174.654226980345


In [610]:
# Set up for AIC and BIC function
mod_col4 = X4.columns.tolist()
col_range4 = range(len(mod_col4))
AIC_list4 = []
BIC_list4 = []

In [611]:
# Calculate AIC and BIC for possible models
for col in col_range4:
    X5 = X4.drop(columns = mod_col4[col])
    X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y, random_state = 1)
    y5_test = y5_test.reset_index(drop = True)
    mlr_model5 = linear_model.LinearRegression()
    mlr_model5.fit(X5_train, y5_train)
    y5_pred = mlr_model5.predict(X5_test)
    residuals5 = y5_test - y5_pred
    mse = stat.mean(residuals5 ** 2)
    n = len(residuals5)
    k = (len(mlr_model5.coef_) + 1)
    aic = (2*k + n*math.log(mse) + n*math.log(2*math.pi) + n)
    bic = (k*math.log(n) + n*math.log(mse) + n*math.log(2*math.pi) + n)
    AIC_list4.append(aic)
    BIC_list4.append(bic)

In [612]:
# Display AIC and BIC scores to determine which variable to drop
print(AIC_list4)
print(BIC_list4)

[15211.182307637535, 15161.171598345509]
[15220.012501555879, 15170.001792263853]


In [572]:
# Drop variable with the lowest AIC and BIC
X5 = X4.drop(columns = ['Company Size'])
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y, random_state = 1)
y5_test = y5_test.reset_index(drop = True)
mlr_model5 = linear_model.LinearRegression()
mlr_model5.fit(X5_train, y5_train)
y5_pred = mlr_model5.predict(X5_test)
residuals5 = y5_test - y5_pred

In [573]:
# Create predictions and residuals dataframe
pred5_df = pd.DataFrame(y5_pred, columns = ['Predictions'])
pred5_df['Testing Data'] = y5_test
pred5_df['Residuals'] = residuals5
pred5_df

,Predictions,Testing Data,Residuals
0,169138.998674,140000,-29138.998674
1,100533.946289,90000,-10533.946289
2,169138.998674,247500,78361.001326
3,169138.998674,123648,-45490.998674
4,169138.998674,311000,141861.001326
...,...,...,...
606,134836.472482,106500,-28336.472482
607,169138.998674,225000,55861.001326
608,169138.998674,149850,-19288.998674
609,169138.998674,165000,-4138.998674


In [575]:
# Calculate the R^2 value and display residuals vs. fitted plot
print(r2_score(y5_test, y5_pred))
pred5_df.hvplot.scatter(x = 'Predictions', y = 'Residuals', title = 'Residuals vs. Fitted Plot')

0.07902209961920414


:Scatter   [Predictions]   (Residuals)

## Logistic Regression with Job Title as the Independent Variable

In [328]:
jt_df = new_df.drop(columns = ['Employment Type', 'Expertise Level', 'Salary Currency', 'Company Location', 'Salary in USD'])
jt_df

,Job Title,Experience Level,Salary,Employee Residence,Company Size,Year
0,Data Engineer,Senior,210000,United States,Medium,2023
1,Data Engineer,Senior,165000,United States,Medium,2023
2,Data Engineer,Senior,185900,United States,Medium,2023
3,Data Engineer,Senior,129300,United States,Medium,2023
4,Data Scientist,Senior,140000,United States,Medium,2023
...,...,...,...,...,...,...
2470,Applied Machine Learning Scientist,Mid,423000,United States,Large,2021
2471,Data Specialist,Senior,165000,United States,Large,2021
2472,Data Scientist,Senior,412000,United States,Large,2020
2473,Principal Data Scientist,Mid,151000,United States,Large,2021


In [205]:
# # Split job titles to look at last word in job title
# len_jt = range(2475)

# for i in len_jt:
#     last_word = jt_df['Job Title'][i].split()[-1]
#     jt_df.at[i, 'Job Title'] = last_word

In [329]:
titles_count2 = jt_df['Job Title'].value_counts()
titles_count2

Data Engineer                          557
Data Scientist                         469
Data Analyst                           359
Machine Learning Engineer              215
Analytics Engineer                     109
                                      ... 
Business Intelligence Data Analyst       1
Principal Machine Learning Engineer      1
Compliance Data Analyst                  1
Manager Data Management                  1
Principal Data Analyst                   1
Name: Job Title, Length: 87, dtype: int64

In [330]:
# Choose a cutoff value and create a list of job titles to be replaced
# use the variable name `residence_to_replace`
cutoff_value = 30
other_jt_df = jt_df

titles_to_replace2 = titles_count2[titles_count2 < cutoff_value].index.tolist()

# Replace in dataframe
for jt in titles_to_replace:
    other_jt_df['Job Title'] = jt_df['Job Title'].replace(jt, "Other")

# Check to make sure binning was successful
other_jt_df['Job Title'].value_counts()

Data Engineer                 557
Data Scientist                469
Other                         380
Data Analyst                  359
Machine Learning Engineer     215
Analytics Engineer            109
Research Scientist             81
Data Architect                 76
Data Science Manager           53
ML Engineer                    48
Applied Scientist              48
Research Engineer              46
Machine Learning Scientist     34
Name: Job Title, dtype: int64

In [331]:
# Check to see how many employees are located outside of the United States
residence_counts2 = jt_df['Employee Residence'].value_counts()

In [332]:
# Choose a cutoff value and create a list of countries of residence to be replaced
# use the variable name `residence_to_replace`
cutoff_value = 1000

residence_to_replace = residence_counts2[residence_counts2 < cutoff_value].index.tolist()

# Replace in dataframe
for res in residence_to_replace:
    jt_df['Employee Residence'] = jt_df['Employee Residence'].replace(res, "Other")

# Check to make sure binning was successful
jt_df['Employee Residence'].value_counts()

United States    2437
Other              38
Name: Employee Residence, dtype: int64

In [333]:
# Code experience levels into numeric values
jt_df.loc[jt_df['Experience Level'] == 'Entry', 'Experience Level'] = 0
jt_df.loc[jt_df['Experience Level'] == 'Mid', 'Experience Level'] = 1
jt_df.loc[jt_df['Experience Level'] == 'Senior', 'Experience Level'] = 2
jt_df.loc[jt_df['Experience Level'] == 'Executive', 'Experience Level'] = 3

In [334]:
# Code employee residence into numeric values
jt_df.loc[jt_df['Employee Residence'] == 'United States', 'Employee Residence'] = 0
jt_df.loc[jt_df['Employee Residence'] == 'Other', 'Employee Residence'] = 1

In [335]:
# Code company size into numeric values
jt_df.loc[jt_df['Company Size'] == 'Small', 'Company Size'] = 0
jt_df.loc[jt_df['Company Size'] == 'Medium', 'Company Size'] = 1
jt_df.loc[jt_df['Company Size'] == 'Large', 'Company Size'] = 2

In [336]:
jt_df.to_csv('../Data/Job_Titles_Clean_Data_Science_Salaries.csv', index=False)

In [337]:
# Load the dataset
file_path2 = "../Data/Job_Titles_Clean_Data_Science_Salaries.csv"
new_jt_df = pd.read_csv(file_path2, index_col = 'Job Title')

new_jt_df

,Experience Level,Salary,Employee Residence,Company Size,Year
Job Title,,,,,
Data Engineer,2,210000,0,1,2023
Data Engineer,2,165000,0,1,2023
Data Engineer,2,185900,0,1,2023
Data Engineer,2,129300,0,1,2023
Data Scientist,2,140000,0,1,2023
...,...,...,...,...,...
Other,1,423000,0,2,2021
Other,2,165000,0,2,2021
Data Scientist,2,412000,0,2,2020


In [338]:
# Plot your data to see what's in your DataFrame
new_jt_df.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [Job Title]   (value)

In [339]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
new_jt_data_scaled = StandardScaler().fit_transform(new_jt_df)

In [340]:
# Create a DataFrame with the scaled data
new_jt_df_scaled = pd.DataFrame(new_jt_data_scaled, columns = new_jt_df.columns, index = new_jt_df.index)
new_jt_df_scaled

,Experience Level,Salary,Employee Residence,Company Size,Year
Job Title,,,,,
Data Engineer,0.408328,0.181058,-0.124872,-0.236117,0.661946
Data Engineer,0.408328,-0.003416,-0.124872,-0.236117,0.661946
Data Engineer,0.408328,0.082262,-0.124872,-0.236117,0.661946
Data Engineer,0.408328,-0.149766,-0.124872,-0.236117,0.661946
Data Scientist,0.408328,-0.105902,-0.124872,-0.236117,0.661946
...,...,...,...,...,...
Other,-1.134590,1.054236,-0.124872,2.791814,-2.531653
Other,0.408328,-0.003416,-0.124872,2.791814,-2.531653
Data Scientist,0.408328,1.009143,-0.124872,2.791814,-4.128452


In [341]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [342]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    jt_model = KMeans(i, random_state = 0)
    jt_model.fit(new_jt_df_scaled)
    inertia.append(jt_model.inertia_)

In [343]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_dict)

In [344]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(x = "k", y = "inertia", title = "Elbow Curve", xticks = k)

:Curve   [k]   (inertia)

In [345]:
# Initialize the K-Means model using the best value for k
new_jt_model = KMeans(n_clusters = 6, random_state = 0)

In [346]:
# Fit the K-Means model using the scaled data
new_jt_model.fit(new_jt_df_scaled)

KMeans(n_clusters=6, random_state=0)

In [347]:
# Predict the clusters to group the cryptocurrencies using the scaled data
jt_predict = new_jt_model.predict(new_jt_df_scaled)

# Print the resulting array of cluster values.
print(jt_predict)

[0 0 0 ... 2 2 1]


In [348]:
# Create a copy of the DataFrame
new_jt_predict_df_scaled = new_jt_df_scaled.copy()

In [349]:
# Add a new column to the DataFrame with the predicted clusters
new_jt_predict_df_scaled["predicted_clusters"] = jt_predict

# Display sample data
new_jt_predict_df_scaled.head()

,Experience Level,Salary,Employee Residence,Company Size,Year,predicted_clusters
Job Title,,,,,,
Data Engineer,0.408328,0.181058,-0.124872,-0.236117,0.661946,0
Data Engineer,0.408328,-0.003416,-0.124872,-0.236117,0.661946,0
Data Engineer,0.408328,0.082262,-0.124872,-0.236117,0.661946,0
Data Engineer,0.408328,-0.149766,-0.124872,-0.236117,0.661946,0
Data Scientist,0.408328,-0.105902,-0.124872,-0.236117,0.661946,0


In [351]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
new_jt_predict_df_scaled.hvplot.scatter(x = "Salary", y = "Company Size", by = "predicted_clusters", hover_cols = "Job Title")


:NdOverlay   [predicted_clusters]
   :Scatter   [Salary]   (Company Size,Job Title)